# CTA Data Analysis - ETL
---
## Outline
This is part one of a two part analysis of the CTA Ridership data. In this section, I will focus on pulling the data from the CTA website via API, cleaning and standardizing the data, and then loading the data to a SQLite database. 

1. Pull CTA Data Using API (Extract)
2. Clean and Standardize Data (Transform)
3. Normalize Data (Transform)
4. Store Data in SQLite (Load)


In [1]:
import pandas as pd
import sodapy

# remove Socrata API warnings about unauthenticated cleints
import warnings
warnings.simplefilter('ignore')


## Pull CTA Data Using API (Extract)
Socrata hosts over one hundred different data catalogs for governments, non-profits, and NGOs around the world, which can be easily pulled via API. Every Socrata open dataset has a built-in SODA API. I'm going to be using an unauthenticated client, which only works with public data sets. 

Since I'm using an unauthenticated client, sodapy defaults my reuslts to 1,000 rows. However, if I set the limit myself I can avoid that. The CTA documentation states the max of rows of the data set is 11,593, which is why I set the limit to 12,000.

In [2]:
# unauthenticated client only works with public data sets
# Note 'None' in place of application token, and no username or password
client = sodapy.Socrata("data.cityofchicago.org", None)


In [3]:
# dataset identifier: 5eg2-c264 
# client returned as JSON from API converted to Python list of dictionaries by sodapy
cta_data = client.get("5eg2-c264", limit=12000)


In [4]:
# convert to pandas DataFrame using from_records 
cta_df = pd.DataFrame.from_records(cta_data)

# view the first 5 rows of the data 
cta_df.head()


,alightings,boardings,cross_street,daytype,location,month,on_street,routes,stop_id
0,150.0,183.4,AUSTIN,Weekday,"{'type': 'Point', 'coordinates': [-87.77410482...",2012-10-01T00:00:00.000,JACKSON,126,1
1,0.2,5.3,MAYFIELD (EXTENDED),Weekday,"{'type': 'Point', 'coordinates': [-87.77131794...",2012-10-01T00:00:00.000,JACKSON,126,2
2,0.7,8.3,MENARD,Weekday,"{'type': 'Point', 'coordinates': [-87.76975039...",2012-10-01T00:00:00.000,JACKSON,126,3
3,3.0,17.9,5700 WEST,Weekday,"{'type': 'Point', 'coordinates': [-87.76745055...",2012-10-01T00:00:00.000,JACKSON,126,4
4,11.2,74.0,LOTUS,Weekday,"{'type': 'Point', 'coordinates': [-87.761446, ...",2012-10-01T00:00:00.000,JACKSON,126,6


### Describe the shape and data types of dataframe
Since the data is unknown to me, I need to do basic analysis. For example, what's the shape of my data and what are the data types of each column?   

In [5]:
# describe the shape of dataframe
cta_df.shape


(11593, 9)

In [6]:
# understand the data types
cta_df.dtypes


alightings      object
boardings       object
cross_street    object
daytype         object
location        object
month           object
on_street       object
routes          object
stop_id         object
dtype: object

### Data dictionary of columns
CTA - Ridership - Avg. Weekday Bus Stop Boardings in October 2012
This dataset shows approximate, average, weekday boardings by bus stop from the month of October 2012.

* `stop_id:` unique identifier of bus stops
* `on_street:` name of street where stop is located on
* `cross_street:` name of street crossing with or connecting to street where stop is located
* `routes:` identifier of bus routes
* `month_beginning:`date of first of the month
* `day_type:` name of day ridership was reported on  
* `location:` point coordinates of stop location
* `boardings:` number of passengers boarding a bus 
* `alightings:`number of passengers exiting a bus 

### Write dataframe to cvs file for further investigation in Excel
I need to decide what’s important for the analysis, and I want to get familiar with the data. Sometimes working with Excel is necessary for gaining a better understanding of a data set. My main purpose for using excel is to understand how I should clean and structure my data.

In [7]:
# write data to csv file 
cta_df.to_csv('data/CTA_Ridership_October_2012.csv')


## Clean and Standardize Data (Transform)
While I was investigating the data in Excel, I noticed there are discrepandies between the data set returned with the API and the online data set. Below I've listed out the following steps I'm going to take to clean the data:

1. Find any null values, and remove them if necessary
2. Rename columns `daytype` and `month` into the following: `day_type` and `month_beginnging`
3. Transform `location` column from a dictionary into seperate columns of `longitude` and `latitude`
4. Make `day_type` column uppercase (match the same string format as other columns)
6. Set the data types of the columns
7. Reorder columns to match the online data set

### Step 1: Find any null values

In [8]:
# Use isnull and any to build a boolean Series and use that to index my frame
cta_df[cta_df.isnull().any(axis=1)]


,alightings,boardings,cross_street,daytype,location,month,on_street,routes,stop_id
6642,35.2,62.4,KEELER,Weekday,"{'type': 'Point', 'coordinates': [-87.73212, 4...",2012-10-01T00:00:00.000,BELMONT,NaN,9267
9211,185.5,2.1,LAKE SHORE,Weekday,"{'type': 'Point', 'coordinates': [-87.64355208...",2012-10-01T00:00:00.000,ADDISON,NaN,12548


I'm going to drop the two rows beause they're missing a fundamental piece of data. How can you have 62.4 boardings and 35.2 alightings when you don't even have a bus? I also don't need them to answer the questions below and they might cause a conflict when I'm doing further analysis. For example,  on `alightings` and `boardings` columns.

Questions:
1. What is the route with the most stops?
2. What is the stop with the most routes?

In [9]:
# drop the two rows and reset the index
cta_df = cta_df.dropna()  
cta_df = cta_df.reset_index(drop=True)

# print the shape of the dataframe after dropping rows with NaN
cta_df.shape


(11591, 9)

### Step 2: Rename columns

In [10]:
# rename column daytype to day_type
cta_df = cta_df.rename({'daytype':'day_type', 
                        'month':'month_beginning'}, axis='columns')


### Step 3: Transform location into longitude and latitude columns

I need to transform the location column from a dictionary into seperate columns of longitude and latitude. Note, the API documentation on the field: 
> Heads up! Contrary to the normal convention of "latitude, longitude" orderingin the coordinates property, order the coordinates as "longitude, latitude"

In [11]:
# transform location column from dictionary 
# into columns of longitude, latitude
cta_df['location'] = cta_df['location'].apply(lambda x: x['coordinates'])

# create dataframe of locations split into longitude and latitude columns
coords_df = pd.DataFrame(cta_df.location.values.tolist(), 
                         columns=['longitude','latitude'], 
                         index=cta_df.index, 
                         dtype=str)

# coords_df join back into main cta_df dataframe using the same index
cta_df = pd.merge(cta_df, coords_df, 
                  right_index=True, 
                  left_index=True)

# drop the location column since it's split into longitude and latitude
cta_df = cta_df.drop(['location'], axis=1)

cta_df.head()


,alightings,boardings,cross_street,day_type,month_beginning,on_street,routes,stop_id,longitude,latitude
0,150.0,183.4,AUSTIN,Weekday,2012-10-01T00:00:00.000,JACKSON,126,1,-87.77410482,41.87632184
1,0.2,5.3,MAYFIELD (EXTENDED),Weekday,2012-10-01T00:00:00.000,JACKSON,126,2,-87.77131794,41.87706679
2,0.7,8.3,MENARD,Weekday,2012-10-01T00:00:00.000,JACKSON,126,3,-87.76975039,41.87695725
3,3.0,17.9,5700 WEST,Weekday,2012-10-01T00:00:00.000,JACKSON,126,4,-87.76745055,41.87702418
4,11.2,74.0,LOTUS,Weekday,2012-10-01T00:00:00.000,JACKSON,126,6,-87.761446,41.876513


### Step 4: Make day_type column uppercase

In [12]:
# convert day_type to uppercase to match string format across columns
cta_df['day_type'] = cta_df['day_type'].str.upper()
cta_df.head()


,alightings,boardings,cross_street,day_type,month_beginning,on_street,routes,stop_id,longitude,latitude
0,150.0,183.4,AUSTIN,WEEKDAY,2012-10-01T00:00:00.000,JACKSON,126,1,-87.77410482,41.87632184
1,0.2,5.3,MAYFIELD (EXTENDED),WEEKDAY,2012-10-01T00:00:00.000,JACKSON,126,2,-87.77131794,41.87706679
2,0.7,8.3,MENARD,WEEKDAY,2012-10-01T00:00:00.000,JACKSON,126,3,-87.76975039,41.87695725
3,3.0,17.9,5700 WEST,WEEKDAY,2012-10-01T00:00:00.000,JACKSON,126,4,-87.76745055,41.87702418
4,11.2,74.0,LOTUS,WEEKDAY,2012-10-01T00:00:00.000,JACKSON,126,6,-87.761446,41.876513


### Step 5: Set column data types

In [13]:
# convert month_beginning to datetime object
cta_df['month_beginning'] = pd.to_datetime(cta_df['month_beginning'])

# convert columns proper data types
cta_df = cta_df.astype({'alightings':float, 
                        'boardings':float,
                        'stop_id':int,
                        'longitude':float,
                        'latitude':float})

# print the data types after converting them
cta_df.dtypes


alightings                float64
boardings                 float64
cross_street               object
day_type                   object
month_beginning    datetime64[ns]
on_street                  object
routes                     object
stop_id                     int64
longitude                 float64
latitude                  float64
dtype: object

### Step 6: Reorder columns

In [14]:
# reorder columns in dataframe
cta_df = cta_df[['stop_id', 'on_street', 
                 'cross_street', 'routes',
                 'boardings', 'alightings', 
                 'month_beginning', 'day_type',
                 'longitude', 'latitude']]
cta_df.head()


,stop_id,on_street,cross_street,routes,boardings,alightings,month_beginning,day_type,longitude,latitude
0,1,JACKSON,AUSTIN,126,183.4,150.0,2012-10-01,WEEKDAY,-87.774105,41.876322
1,2,JACKSON,MAYFIELD (EXTENDED),126,5.3,0.2,2012-10-01,WEEKDAY,-87.771318,41.877067
2,3,JACKSON,MENARD,126,8.3,0.7,2012-10-01,WEEKDAY,-87.769750,41.876957
3,4,JACKSON,5700 WEST,126,17.9,3.0,2012-10-01,WEEKDAY,-87.767451,41.877024
4,6,JACKSON,LOTUS,126,74.0,11.2,2012-10-01,WEEKDAY,-87.761446,41.876513


## Normalize Data (Transform)
While investigating the data, I noticed the data set was not normalized. At a glance there’s not a lot wrong with the data. It’s exactly what you would expect to see in a *CTA Ridership Avgerage Weekday Bus Stop Boardings Report*. However, it’s not how we should be storing data. There are a number of reasons why. Part of it is just efficiency, part of it is simplifying the queries, and the other part is to minize or avoid data modification issues.

Database normalization is process used to organize a database into tables and columns. The idea is that a table should be about a specific topic and that only those columns which support that topic are included. 

One of the main reasons for normalizing the data set was how the `routes` information was stored. Routes were concatenated together for every stop. In order to simplify my queries and minimize or avoid data modification issues, I split (or exploded) every route and stop into discrete values. For example, instead of `stop_id=10647` and `routes=12,10,122,123,124`, I split all of the routes into seperate rows with `stop_id`. 

I'm going to split the data into three tables: 
1. `bus_stops:` Main table consisting of bus stop's street, cross street, and coordinates.
2. `bus_routes:` Individual bus routes and stops; references `bus_stops` table by `stop_id`.
3. `bus_ridership:`Bus boardings and alightings for each stop on a weekday basis; references `bus_stops` table by `stop_id`.

### Bus stops table

In [15]:
# create ridership table from cta_df
bus_stops_df = cta_df.drop(columns=['routes', 'boardings', 
                                    'alightings', 'month_beginning',
                                    'day_type'])

# set stops dataframe index as stop_id
bus_stops_df = bus_stops_df.set_index('stop_id')

bus_stops_df.head()


,on_street,cross_street,longitude,latitude
stop_id,,,,
1,JACKSON,AUSTIN,-87.774105,41.876322
2,JACKSON,MAYFIELD (EXTENDED),-87.771318,41.877067
3,JACKSON,MENARD,-87.769750,41.876957
4,JACKSON,5700 WEST,-87.767451,41.877024
6,JACKSON,LOTUS,-87.761446,41.876513


### Bus routes table

In [16]:
# create a copy of the dataframe to explode the routes column into seperate rows
bus_routes_df = cta_df.copy() 
bus_routes_df = bus_routes_df.drop(columns=['on_street', 'cross_street',
                                            'boardings', 'alightings', 
                                            'month_beginning', 'day_type',
                                            'longitude', 'latitude'])

# explode routes column into discrete rows
bus_routes_df = pd.DataFrame(bus_routes_df.routes.str.split(',').tolist(),
                             index=bus_routes_df.stop_id).stack()

# reset index and rename 0 to stop_id
bus_routes_df = bus_routes_df.reset_index()[[0, 'stop_id']] 
bus_routes_df.columns = ['route', 'stop_id']

# set routes dataframe index name as id
bus_routes_df.index.name = 'id'

# reorder columns
bus_routes_df = bus_routes_df[['stop_id','route']]

# print the new dataframe shape and head
print(bus_routes_df.shape)
bus_routes_df.head()


(15271, 2)


,stop_id,route
id,,
0,1,126
1,2,126
2,3,126
3,4,126
4,6,126


### Bus ridership table

In [17]:
# create ridership table from cta_df
bus_ridership_df = cta_df.drop(columns=['routes', 'on_street',
                                        'cross_street', 'longitude', 
                                        'latitude'])

# set ridership dataframe index name as id
bus_ridership_df.index.name = 'id'

# print top rows
bus_ridership_df.head()


,stop_id,boardings,alightings,month_beginning,day_type
id,,,,,
0,1,183.4,150.0,2012-10-01,WEEKDAY
1,2,5.3,0.2,2012-10-01,WEEKDAY
2,3,8.3,0.7,2012-10-01,WEEKDAY
3,4,17.9,3.0,2012-10-01,WEEKDAY
4,6,74.0,11.2,2012-10-01,WEEKDAY


## Store Data in SQLite (Load)
I'm going to storing my data in a SQLite3 database. First, I will create the tables in the database. Then I will load the data into the tables. 

**Database model:**
![title](database/cta_bus_ridership_database_model.png)

In [18]:
import sqlite3

# helper functions for sqlite3 
def connect(sqlite_file):
    """
    Make connection to an SQLite database file.
    """
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    return conn, c

def close(conn):
    """ 
    Commit changes and close connection to the database.
    """
    conn.close()


In [19]:
# sql_db is my connection and sql_command is my cursor for sqlite3
sql_db, sql_command = connect('database/cta_bus_ridership.db')


### Create tables in database

In [20]:
# sql create command for tables
cta_bus_ridership_tables = """CREATE TABLE bus_stops (
                                stop_id INTEGER PRIMARY KEY,
                                on_street VARCHAR(30),
                                cross_street VARCHAR(30),
                                longitude REAL, 
                                latitude REAL
                                );
                                
                            CREATE TABLE bus_routes (
                                id INTEGER PRIMARY KEY,
                                stop_id INTEGER,
                                route VARCHAR(20),
                                FOREIGN KEY(stop_id) REFERENCES bus_stops(stop_id)
                                );
                                
                            CREATE TABLE bus_ridership (
                                id INTEGER PRIMARY KEY,
                                stop_id INTEGER,
                                boardings REAL,
                                alightings REAL,
                                month_beginning DATETIME, 
                                day_type VARCHAR(15),
                                FOREIGN KEY(stop_id) REFERENCES bus_stops(stop_id)
                                );"""

sql_command.executescript(cta_bus_ridership_tables)


### Load dataframes into database tables

In [21]:
# load dataframes into sql tables
bus_stops_df.to_sql(name='bus_stops', con=sql_db,
                    if_exists='append', index=True)

bus_routes_df.to_sql(name='bus_routes', con=sql_db,
                     if_exists='append', index=True)

bus_ridership_df.to_sql(name='bus_ridership', con=sql_db,
                        if_exists='append', index=True)


## See you in CTA Analysis - Part 2 !
Now that my data is clean and normalized in a SQLite database, I'm ready to build a story and produce insights of data. 